In [2]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [3]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/lucy/processed/metadata')
db_path=os.path.join(root_dir,'db/lucy.db')

In [4]:
#define input tables and metadata
tbl_names_lst=["b_sample","l_sample"]

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

In [5]:
#setup parameters
DFLT_PARAMS={
    'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
    'LOGIC_OPS':{'AND':0.9,'OR':0.1},
    'NOT_STATE':{'0':0.8, '1':0.2},
    'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.15, 'IN':0.15, 'NOT LIKE':0.1, 'NOT IN':0.1},
    'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.2, 'NOT BETWEEN':0.1},
    'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.2, 'IN':0.1, 'NOT BETWEEN':0.1, 'NOT IN':0.1},
    'FILTER_TYPE':{'WHERE':0.5, 'AND':0.5}, #Use WHERE or AND with JOIN CLAUSE
    'JOIN_TYPE': {'JOIN':0.5, 'LEFT JOIN':0.5}
}

#connect to db
conn = sqlite3.connect(db_path)

self=RND_QUERY(conn, tbl_names_lst, metadata_lst,DFLT_PARAMS)

Metadata fot table b_sample validated.
Metadata fot table l_sample validated.


# Generate Random  queries 

In [6]:
#Single Aggregate 
rnd_query=self.make_single_agg_query(agg_fntn=True)

n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

SELECT b_sample.NEOMAT, COUNT(*), MIN(l_sample.MDC)  FROM b_sample  JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  GROUP BY b_sample.NEOMAT
no_of_rows=3
no_of_cols=3
product=0.009


SELECT b_sample.NEOMAT, COUNT(*), MIN(l_sample.MDC)  FROM b_sample  JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  GROUP BY b_sample.NEOMAT


In [7]:
#Twin Aggregate 
syn_tbl_name_lst=['b_sample_syn_01','l_sample_syn_01']
rnd_query=self.make_twin_agg_query(syn_tbl_name_lst,agg_fntn=True)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")
print(rnd_query['query_desc']['sql_real'])
print("\n")
print(rnd_query['query_desc']['sql_syn'])

REAL:
no_of_rows=27472
no_of_cols=12
product=329.664


SYN:
no_of_rows=30477
no_of_cols=12
product=365.724


SELECT b_sample.PL_RUCC2, b_sample.PL_CBSA, b_sample.HOSPBRTH, b_sample.PL_UR_CA, b_sample.DIED, b_sample.PL_UIC20, b_sample.Homeless, b_sample.PL_NCHS2, l_sample.DX1, b_sample.DNR, COUNT(*), MAX(l_sample.AWEEKEND)  FROM b_sample  LEFT JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  GROUP BY b_sample.PL_RUCC2, b_sample.PL_CBSA, b_sample.HOSPBRTH, b_sample.PL_UR_CA, b_sample.DIED, b_sample.PL_UIC20, b_sample.Homeless, b_sample.PL_NCHS2, l_sample.DX1, b_sample.DNR


SELECT b_sample_syn_01.PL_RUCC2, b_sample_syn_01.PL_CBSA, b_sample_syn_01.HOSPBRTH, b_sample_syn_01.PL_UR_CA, b_sample_syn_01.DIED, b_sample_syn_01.PL_UIC20, b_sample_syn_01.Homeless, b_sample_syn_01.PL_NCHS2, l_sample_syn_01.DX1, b_sample_syn_01.DNR, COUNT(*), MAX(l_sample_syn_01.AWEEKEND)  FROM b_sample_syn_01  LEFT JOIN l_sample_syn_01  ON b_sample_syn_01.PNUM_R=l_sample_syn_01.PNUM_R  GROUP BY b_sample_syn_01.PL

In [8]:
#Single Filter

# self.max_in_terms=2
# self.no_groupby_vars=2
self.no_where_vars=2
# self.no_join_tables=1

rnd_query=self.make_single_fltr_query()


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

SELECT *  FROM b_sample  LEFT JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  WHERE   l_sample.ASCHED LIKE '1' AND  b_sample.FEMALE NOT LIKE '1' 
no_of_rows=90641
no_of_cols=31
product=2809.871


SELECT *  FROM b_sample  LEFT JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  WHERE   l_sample.ASCHED LIKE '1' AND  b_sample.FEMALE NOT LIKE '1' 


In [9]:
# Twin Filter 

self.no_where_vars=2


syn_tbl_name_lst=['b_sample_syn_01','l_sample_syn_01']
rnd_query=self.make_twin_fltr_query(syn_tbl_name_lst)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")
print(rnd_query['query_desc']['sql_real'])
print("\n")
print(rnd_query['query_desc']['sql_syn'])

SELECT *  FROM b_sample  LEFT JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  WHERE   b_sample.HISPANIC = '2' AND  l_sample.NPR > 2 
REAL:
no_of_rows=153182
no_of_cols=31
product=4748.642


SYN:
no_of_rows=151490
no_of_cols=31
product=4696.19


SELECT *  FROM b_sample  LEFT JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  WHERE   b_sample.HISPANIC = '2' AND  l_sample.NPR > 2 


SELECT *  FROM b_sample_syn_01  LEFT JOIN l_sample_syn_01  ON b_sample_syn_01.PNUM_R=l_sample_syn_01.PNUM_R  WHERE   b_sample_syn_01.HISPANIC = '2' AND  l_sample_syn_01.NPR > 2 


In [10]:
rnd_query['query_desc']

{'type': 'twin_fltr',
 'from_tbl_name_real': 'b_sample',
 'join_tbl_name_lst_real': ['l_sample'],
 'sql_real': "SELECT *  FROM b_sample  LEFT JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  WHERE   b_sample.HISPANIC = '2' AND  l_sample.NPR > 2 ",
 'n_cols_real': 31,
 'n_rows_real': 153182,
 'from_tbl_name_syn': 'b_sample_syn_01',
 'join_tbl_name_lst_syn': ['l_sample_syn_01'],
 'sql_syn': "SELECT *  FROM b_sample_syn_01  LEFT JOIN l_sample_syn_01  ON b_sample_syn_01.PNUM_R=l_sample_syn_01.PNUM_R  WHERE   b_sample_syn_01.HISPANIC = '2' AND  l_sample_syn_01.NPR > 2 ",
 'n_cols_syn': 31,
 'n_rows_syn': 151490}

In [11]:
''' This query resulted in kernel crash 
SELECT *  FROM b_sample  JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  AND   b_sample.Homeless IN ('0', '0', '0') AND  l_sample.TOTCHG >= 85649 AND  l_sample.HCUP_ED = 0 AND  l_sample.PROCTYPE <= 1 AND  l_sample.DX1 = '515' AND  b_sample.NEOMAT IN ('0', '0', '0') AND  l_sample.Date > 6 AND  b_sample.RACE LIKE '1' AND NOT  l_sample.ASCHED IN ('0', '0', '0') OR  b_sample.PL_RUCA4 <> '1' AND  l_sample.NPR <> 3 AND  l_sample.NECODE < 2 AND  l_sample.AWEEKEND > 0 AND NOT  b_sample.DIED LIKE '0' AND  b_sample.PL_UIC20 LIKE '1' AND  b_sample.PL_CBSA <> '2' AND  b_sample.DNR IN ('0', '0', '0') AND  b_sample.HISPANIC IN ('2', '1', '1') OR  b_sample.PL_RUCC2 IN ('1', '1', '1') 

'''

" This query resulted in kernel crash \nSELECT *  FROM b_sample  JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  AND   b_sample.Homeless IN ('0', '0', '0') AND  l_sample.TOTCHG >= 85649 AND  l_sample.HCUP_ED = 0 AND  l_sample.PROCTYPE <= 1 AND  l_sample.DX1 = '515' AND  b_sample.NEOMAT IN ('0', '0', '0') AND  l_sample.Date > 6 AND  b_sample.RACE LIKE '1' AND NOT  l_sample.ASCHED IN ('0', '0', '0') OR  b_sample.PL_RUCA4 <> '1' AND  l_sample.NPR <> 3 AND  l_sample.NECODE < 2 AND  l_sample.AWEEKEND > 0 AND NOT  b_sample.DIED LIKE '0' AND  b_sample.PL_UIC20 LIKE '1' AND  b_sample.PL_CBSA <> '2' AND  b_sample.DNR IN ('0', '0', '0') AND  b_sample.HISPANIC IN ('2', '1', '1') OR  b_sample.PL_RUCC2 IN ('1', '1', '1') \n\n"